# CART

Б). Реализовать обучение и вычисление дерева с использованием алгоритма CART для задачи регрессии и задачи классификации. Выполнить оценку качества моделей, визуализировать дерево решений, вывести решающие правила.

## Теория

Алгоритм CART
Строит бинарное дерево, где в узлах находится предикат, в листах находится ответ.
Задача -- минимизировать ошибку на каждом листе.

Алгоритм обучения упрощённо можно описать следующим образом:

0. Проверяем критерий остановки
1. Строим всевозможные разбиения на две подвыборки по одному признаку
2. Выбираем лучшее разбиение
3. Возвращаемся к шагу 0 для потомков
4. Проводим отсечение (pruning)

\begin{align*}
IG = S_0 - \sum_{i=1}^q \frac{N_i}{N}S_i - Энтропия
\end{align*}

$q$ - количество подвыборок (в бинарном дереве две)

$IG$ - информационный выигрыш

$S_0$ - начальная энтропия

$N_i$ - элементы до порога

$N$ - все элементы выборки

$S_i$ - энтропия элементов

\begin{align*}
S = - \sum_{i=1}^N p_i \log_2{p_i}
\end{align*}

$N$ - количество классов

## Реализация

In [5]:
import numpy as np
from collections import Counter

In [30]:
class CART:

    def __init__(self,
                 max_depth: int | None = None, 
                 min_samples_split: int = 2,
                 classification: bool = False,
                 ) -> None:
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.classification = classification

    class Node:
        def __init__(self, 
                     feature: int | None = None, 
                     threshold: float | None = None, 
                     value: int | float | None = None, 
                     left = None, right = None
                     ) -> None:
                self.feature = feature
                self.threshold = threshold
                self.value = value
                self.left = left
                self.right = right

    def _entropy(self, Y: np.ndarray) -> float:
        """Находит энтропию столбца"""
        probabilities = np.array(list(Counter(Y).values())) / len(Y)
        return -np.sum(probabilities * np.log2(probabilities))
    
    def _MSE(self, Y: np.ndarray) -> float:
        """Находит среднеквадратичную ошибку столбца"""
        return np.mean((Y - np.mean(Y))**2)

    def _split_dataset(self, X: np.ndarray, Y: np.ndarray, feature: int, threshold: float):
        """
        Разделяет датасеты на левую и правую подвыборку 
        по признаку feature на основе порога threshold
        """
        left_indexes = np.where(X[:,feature] <= threshold)[0]
        right_indexes = np.where(X[:,feature] > threshold)[0]
        return X[left_indexes], Y[left_indexes], X[right_indexes], Y[right_indexes]

    def _find_best_split(self, X: np.ndarray, Y: np.ndarray):
        """
        Находит лучшее разделение данных на левую и правую подвыборку
        """
        best_feature, best_threshold, best_score = None, None, np.inf

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:,feature])
            for threshold in thresholds:
                x_left, y_left, x_right, y_right = self._split_dataset(X,Y,feature,threshold)

                if self.classification:
                    score = (len(y_left) * self._entropy(y_left) + \
                             len(y_right) * self._entropy(y_right)) / len(Y)
                else:
                    score = (len(y_left) * self._MSE(y_left) + \
                             len(y_right) * self._MSE(y_right)) / len(Y)
                if score < best_score:
                    best_feature, best_threshold, best_score = feature, threshold, score
        return best_feature, best_threshold

    def _build_tree(self, X: np.ndarray, Y: np.ndarray, depth=0) -> Node:
        if depth == self.max_depth or len(X) < self.min_samples_split:
            if self.classification:
                return self.Node(value=Counter(Y).most_common(1)[0][0])
            else:
                return self.Node(value=np.mean(Y))
            
        feature, threshold = self._find_best_split(X,Y)
        x_left, y_left, x_right, y_right = self._split_dataset(X,Y,feature,threshold)
        left_child = self._build_tree(x_left,y_left,depth=depth + 1)
        right_child = self._build_tree(x_right,y_right,depth=depth + 1)

        return self.Node(feature=feature,threshold=threshold,left=left_child,right=right_child)

    def fit(self, X: np.ndarray, Y: np.ndarray):
        self.tree_ = self._build_tree(X,Y)
        return self

    def _predict_single(self, x: np.ndarray, node: Node) -> int | float:
        if node.feature is None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)

    def predict(self, X: np.ndarray) -> list[float] | list[int]:
        y_pred = np.zeros(len(X), dtype=int if self.classification else float)
        for i in range(X.shape[0]):
            y_pred[i] = self._predict_single(X[i], self.tree_)
        return y_pred

## Проверка алгоритма CART

In [31]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x,y = make_classification(
    n_samples=2000,
    n_clusters_per_class=1,
    n_features=4,
    n_classes=3
)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [32]:
from sklearn.metrics import classification_report

crt = CART(classification=True)
crt.fit(x_train,y_train)
predict = crt.predict(x_test)

print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       128
           1       0.90      0.91      0.91       141
           2       0.84      0.82      0.83       131

    accuracy                           0.88       400
   macro avg       0.88      0.88      0.88       400
weighted avg       0.88      0.88      0.88       400



In [35]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)

pred = dt.predict(x_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       128
           1       0.90      0.87      0.89       141
           2       0.81      0.83      0.82       131

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400



In [ ]:
from sklearn.datasets import make_regression
x,y = make_regression(
    n_samples=2000,
    n_features=4
)

x_train, y_train, x_test, y_test = train_test_split(x,y, test_size=0.2)

In [47]:
from my_methods import regression_metrics
crt = CART()
crt.fit(x_train,y_train)
predict = crt.predict(x_test)
regression_metrics('',y_test,predict)

c:\Python\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Python\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



MSE = 0.32
MAE = 0.2
RMSE = 0.565685424949238
MAPE = 258956978573803.56
R^2 = 0.5057485688911199



In [49]:
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)

predict = dt.predict(x_test)
regression_metrics('',y_test,predict)


MSE = 0.27
MAE = 0.175
RMSE = 0.5196152422706632
MAPE = 213920982300098.62
R^2 = 0.5829753550018824

